In [1]:
MISTRAL_MODEL_ENDPOINT = "http://ec2-54-179-34-124.ap-southeast-1.compute.amazonaws.com:8000/v1/completions"
# Transcript - alice is super outdataed, it contains useless emoji

In [ ]:
from openai import OpenAI
from 
client = OpenAI(
    api_key="INSERT_KEY",
)

def get_response(prompt):
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {'role': 'system', 'content': "You are a helpful assistant that always closely follows instructions."},
            {'role': 'user', 'content': prompt}
        ],
    )
    response = completion.choices[0].message.content
    return response

In [5]:
import requests
# from demo import generate_filename, count_tokens, read_transcript
import re
import aiohttp

# Path to transcript, modify here
transcript_dir = "transcripts/Bali-(Aman).txt"
def read_transcript(transcript_dir):
    with open(transcript_dir, 'r', encoding='utf-8') as file:
        transcript = file.read()
    return transcript

conversation_log = read_transcript(transcript_dir=transcript_dir)

# Use the LLM to do the parsing, current version is dumb | and the transcript is not that pretty anyways



In [4]:
conversation_log.split('\n')

['Transcript for Conversation ID: -zwjEMfdLoGbzkZwQOp2W',
 '((2024-04-25 07.28.20.486303)) CUSTOMER: Hello there',
 "((2024-04-25 07.28.33.050240)) AGENT: Hello, Good morning. It's a pleasure to see you this morning. I hope you're feeling well and I'd really like to start off by thanking you for taking the time to meet me today.",
 "((2024-04-25 07.28.34.510309)) CUSTOMER: (smiling) Hi Aman, good morning to you too. I'm doing pretty",
 '((2024-04-25 07.28.37.334718)) CUSTOMER: well, thanks for asking!',
 "((2024-04-25 07.28.38.765250)) CUSTOMER: How about you, how's everything going on your end?",
 "((2024-04-25 07.28.57.326295)) AGENT: I think it's excellent and you know, I've been just so happy with the fact that I've been make been able to make a difference in so many lives in such a short time and I'm hoping I can help you in in any way possible in your future plans.",
 "((2024-04-25 07.28.59.082941)) CUSTOMER: That's great to hear! I've actually been thinking about getting some li

In [8]:
name_mapping = {
    "1. Product_and_Industry_Knowledge": {
        "1.1.1": "Did_AGENT_use_any_wrong_insurance_terms (reverse)",
        "1.1.2": "Did_AGENT_explain_the_value_of_insurance_in_mitigating_risk",
        "1.2.1": "Did AGENT explain coverage of the policy?",
        "1.2.2": "Did AGENT explain features of the policy?",
        "1.2.3": "Did AGENT give any wrong response regarding policy exclusions? (reverse)",
        "1.2.4": "Did AGENT compare product features with those from competitors when necessary?",
        "1.2.5": "Did AGENT explain how the product fits CUSTOMER's unique situation?",
        "1.3.1": "Did AGENT give an example or analogy of how the policy is in line with current social/economic/market situation?",
    },
    "2. Customer Relationship Management": {
        "2.1.1": "Did AGENT open the conversation with professional and friendly greeting?",
        "2.1.2": "Did AGENT initial small talk or show genuine interest in CUSTOMER to build rapport?",
        "2.2.1": "Did AGENT ask questions to elicit sharing about CUSTOMER background?",
        "2.2.2": "Did AGENT ask questions to elicit sharing about CUSTOMER needs?",
        "2.2.3": "Did AGENT do active listening by asking follow-up question about CUSTOMER's background?",
        "2.2.4": "Did AGENT do active listening by paraphrasing or summarizing what CUSTOMER shared?",
        "2.3.1": "Did AGENT demonstrate personalization by referencing customer-specific information previously shared?",
        "2.3.2": "Did AGENT mention that there will be future follow-up or check-in?",
        "2.4.1": "Did AGENT make explicit commitments to service quality and availability for future assistance?",
        "2.4.2": "Did AGENT give an example of how AGENT helped another customer solve a problem in the past?",
        "2.4.3": "Did AGENT explain how CUSTOMER can get help when needed?",
        "2.5.1": "Did AGENT actively ask for feedback?",
        "2.5.2": "Did AGENT respond constructively to feedback?",
    },
    "3. Negotiation and Sales Skills": {
        "3.1.1": "Did AGENT explain USP (Unique Selling Point) of policy compared to competitor?",
        "3.1.2": "Did AGENT link policy benefits with CUSTOMER's particular situation?",
        "3.2.1": "Did AGENT acknowledge all CUSTOMER objections and concerns with empathy?",
        "3.2.2": "Did AGENT propose a solution or alternative to all CUSTOMER objections and concerns?",
        "3.3.1": "Did AGENT check CUSTOMER readiness to buy before the conversation ended?",
        "3.3.2": "Did AGENT summarize key benefits linked to client needs before the conversation ended?",
        "3.4.1": "Did AGENT state the next steps after this conversation?",
        "3.4.2": "Did AGENT ask questions to assess potential to up-sell or cross-sell?",
    },
    "4. Communication Skills": {
        "4.1.1": "Did AGENT use examples or analogies to explain complex insurance terms?",
        "4.1.2": "Did AGENT ask at least one question to check CUSTOMER's understanding?",
        "4.2.1": "Throughout the whole transcript, did AGENT keep explanation succinct without unnecessary details?",
        "4.2.2": "Did AGENT emphasize key points without unnecessary repetition?",
        "4.2.3": "Did AGENT summarize lengthy discussions?",
        "4.3.1": "Did AGENT adjust communication style and language use to CUSTOMER's style and language?",
        "4.3.2": "Did AGENT quickly address all misunderstandings if any?",
        "4.3.3": "Was AGENT sensitive to cultural norms?",
        "4.4.1": "Did AGENT use words like 'I See' or 'I Understand' to show active listening?",
        "4.4.2": "Did AGENT reflect or paraphrase CUSTOMER statements?",
    },
    "5. Analytical Skills": {
        "5.1.1": "Did AGENT point out issues or gaps in CUSTOMER's current insurance coverage if CUSTOMER has existing insurance coverage?",
        "5.1.2": "Did AGENT identify and question assumptions that may affect CUSTOMER's understanding or actions?",
    },
}

In [9]:
PROMPT_TEMPLATE = """<s>[INST]{inst}[/INST]{example}</s><s>[INST]{inst}[/INST]{text}\n\n<OUTPUT>\n\n{query}\n"""

fastModelLevel1PromptGpt = """<task>
- You are a top insurance sales agents in Philippines. 
- You are tasked to review a conversation conducted between a human trainee insurance sales AGENT and a potential CUSTOMER in their first sales meeting and be an assessor. 
- Your overarching objective is to assess the AGENT's performance during the conversation, and provide actionable feedback for AGENT learning.  
- Do this by following the <steps>. Take a deep breath and do this task step by step. 
</task>

<steps>
- Steps to achieve the objective in the <task>: 
- Step 1: for each atom, give one sentence explanation of why it was a "yes" or "no". 
- Step 2: Then give one example of what the AGENT said in the transcript that can be said differently to change a "No" to a "Yes"
</steps>

{additional_information}
Give your feedback in the following format and NOTHING ELSE:

<OUTPUT>

[Numbering for atom] [atom detail]
ASSESSMENT(Change all ASSEASSMENT to ASSESSMENT): [Based on how AGENT did in transcript, RESPOND ONLY WITH "Yes" or "No". There is no "partial" cases.]
REASON: [Reason for giving a "Yes" or "No"]
EXTRACT: [Extract one exact line from transcript that gave a "Yes" or "No"]
IMPROVEMENT: [Provide one example that can be replaced in the conversation transcript that can change a "No" or a "Yes" or a "Yes" to a "No" if atom is reverse]

<END>
"""

EXAMPLE = """
AGENT: Hey there!
CUSTOMER: Hey! Nice to meet you, nice spot for a coffee huh?
AGENT: Yes I like the chill vibe here. So, are you holding any insurance policies in place?
CUSTOMER: Oh I thought we could chat for a bit longer. Yeah I guess I have a policy, but you know its been a while since I looked into it. Do you usually get straight to business with your clients?
AGENT: Oh yes, I always talk business then relax later

<OUTPUT>

1.1.2 Did AGENT explain the value of insurance in mitigating risk?
ASSESSMENT: No 
REASON: When customer talked about policy, you did not provide any information about insurance mitigating risk. You also didn't add it on later on during the conversation so customer does not know the value of your insurance policy comapared to his. 
EXTRACT: "Oh yes, I always talk business then relax later"
IMPROVEMENT: Instead of saying "Oh yes, I always talk business then relax later", maybe you can try saying "FWD's Set for Life is a plan for savings and safety till you're 100. You can adjust it to save for retirement or your kid's school. It lets you invest safely or with risks, and gives you a bonus every five years after the first 10. You can also add more protection for illness or worse cases. It's a way to ensure you're covered for the future!"

<END>
"""

product_info = """
Refer to <productinformation> to check the conversation transcript against policy details. 
<productinformation>
    Companies:
  FWD:
    Product Name: "Set for Life"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "5, 7, 10"
    Currency: "PHP"
    Minimum Premium: "5Pay - PHP 40k, 7Pay - PHP 24k, 10Pay - PHP 20k"
    Minimum SA: "500% of regular annual premium"
    SA Flexibility: "Yes, with SA Multiplier"
    Issue Age: "0 to 70 years old"
    Insurance Coverage: "Up to age 100"
    Death Benefit: "Higher of (SA, AV)"
    Underwriting: "Full Underwriting"
    Mandatory Riders: "Accident, WP"
    Optional Riders: "Term Rider, CI, HIB"
    Commission: "5 Pay - 30%/10%/5%, 7 Pay - 35%/10%/5%, 10 Pay - 40%/15%/5%/5%/5%"
    Premium Charge: "5Pay -70%/45%/0%, 7Pay - 75%/50%/10%/10%, 10Pay - 90%/60%/35%/0%"
    Policy Fee: "n/a"
    Premium Holiday Charge: "None"
    Premium Extension Bonus: "n/a"
    Surrender Charge: "None"

  AXA Life:
    Product Name: "My Life Choice"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "7, 10"
    Currency: "PHP and USD"
    Minimum Premium: "7 pay - PHP 30k, 10 pay - PHP 20k"
    Minimum SA: "7 pay - PHP 210k, 10 pay - PHP 200k"
    SA Flexibility: "Yes, with SA Multiplier"
    Issue Age: "0 to 70 years old"
    Insurance Coverage: "Up to age 100"
    Death Benefit: |
      DB1: AV + SA
      DB2: Higher of (SA, AV)
    Underwriting: "Full Underwriting"
    Mandatory Riders: "Payor's Clause, WP, ADDD"
    Optional Riders: "Term Rider, CI, HIB, Bright Rider"
    Commission: "7 pay - 25%/5%/5%, 10 pay - 35%/10%/5%/5%/5%"
    Premium Charge: "7 Pay - 40%/35%/35%, 10 Pay - 40%/40%/35%/35%/35%"
    Policy Fee: "n/a"
    Premium Holiday Charge: "Same as Premium Charge"
    Premium Extension Bonus: "n/a"
    Surrender Charge: "All Variants"

  Pru Life:
    Product Name: "Prulink Exact Protector"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "5, 7, 10, 15"
    Currency: "PHP and USD"
    Minimum Premium: "PEP -PHP45k / USD600, PEP-LP - PHP 22k"
    Minimum SA: "500% of regular annual premium"
    SA Flexibility: "Yes, with SA Multiplier"
    Issue Age: "0 to 70 years old"
    Insurance Coverage: "Up to age 100"
    Death Benefit: "AV + SA"
    Underwriting: "Full Underwriting"
    Mandatory Riders: "ATPD, ADD"
    Optional Riders: "Future Safe, PA, Payor Term, Payor Waiver, WPTPD, CCB, LCB, LC Plus, LC Advance Plus, Multiple Life Care Plus, LCW, Hospital Income"
    Commission: "5 pay - 35%/15%/5%, 7pay - 35%/15%/15%, 10 pay -  37.50%/15%/5%/5%/5%, 15 pay - 40%/15%/5%/5%/5%"
    Premium Charge: "5 pay - 90%/55%, 7pay - 90%/55%, 10 pay - 93%/70%, 15 pay - 100%/65%/25%"
    Policy Fee: "n/a"
    Premium Holiday Charge: "None"
    Premium Extension Bonus: "n/a"
    Surrender Charge: "None"

  Sunlife Prime:
    Product Name: "Sun MaxiLink Prime"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "10"
    Currency: "PHP"
    Minimum Premium: "PHP 15,225"
    Minimum SA: "PHP350k"
    SA Flexibility: "No"
    Issue Age: "0 to 65 years old"
    Insurance Coverage: "Up to age 88"
    Death Benefit: "AV + SA"
    Underwriting: "Full Underwriting"
    Mandatory Riders: "None"
    Optional Riders: "ADB, TDB, WPD, WPDD, HIB, CIB"
    Commission: ""
    Premium Charge: "Premium Charge: 10 Pay - 65%/5%/5%/5%/5%"
    Policy Fee: "(starting Y2) 10 Pay -40%/20%/5%/5%"
    Premium Holiday Charge: "None"
    Premium Extension Bonus: "n/a"
    Surrender Charge: ""

  Sunlife Bright:
    Product Name: "Sun MaxiLink Bright"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "5"
    Currency: "PHP"
    Minimum Premium: "PHP30.5k"
    Minimum SA: "PHP200k"
    SA Flexibility: "No"
    Issue Age: "0 to 60 years old"
    Insurance Coverage: "Up to age 88"
    Death Benefit: "AV + SA"
    Underwriting: "Full Underwriting"
    Mandatory Riders: "None"
    Optional Riders: "ADB, TDB, WPD, WPDD, HIB, CIB"
    Commission: ""
    Premium Charge: "Premium Charge: 5 Pay - 30%/5%/5%"
    Policy Fee: "(starting Y2) 5 Pay - 15%/5%/5%/5%"
    Premium Holiday Charge: "None"
    Premium Extension Bonus: "n/a"
    Surrender Charge: ""

  Manulife:
    Product Name: "FutureBoost"
    Product Type: "Limited Pay VUL"
    Premium Payment Period: "5, 10"
    Currency: "PHP and USD"
    Minimum Premium: "PHP 50k"
    Minimum SA: "PHP750k"
    SA Flexibility: "Yes, with SA Multiplier"
    Issue Age: "0 to 70 years old"
    Insurance Coverage: "Up to age 99"
    Death Benefit: "Higher of (SA, AV)"
    Underwriting: "Full Underwriting"
    Mandatory Riders: "Accident. WP"
    Optional Riders: "Term, HIB, CI"
    Commission: ""
    Premium Charge: "n/a"
    Policy Fee: "Monthly load 5 pay: Var 1 - 75%/30%, Var 2 - 65%/30%, Var 3 - 30%/10%, 10 pay: Var 1 - 80%/75%/25%, Var 2 - 80%/60%/20%, Var 3 - 65%/25%, 100/month"
    Premium Holiday Charge: "None"
    Premium Extension Bonus: "2%"
    Surrender Charge: "100%/90%/80%/70%/60%/50%/40%/30%"

  </productinformation>
  
"""

In [5]:
class INFERENCE_TYPE:
    GET_NAME = "get_name"
    GET_LEVEL = "get_level"
    CUSTOMER_INTERACTION = "customer_interaction"
    AI_INTERRUPT = "ai_interrupt"


class MistralModel:
    def __init__(
        self,
        customerPromptFastmodel,
        model_path: str = None,
        n_ctx: int = 1024,
        is_api: bool = True,
    ):
        self.model_path = model_path
        self.is_api = is_api
        self.customerPromptFastmodel = customerPromptFastmodel
        self.endpoint = MISTRAL_MODEL_ENDPOINT

    def predict(
        self,
        prompt: str,
        query: str,
        query_additional_info: str = "",
        n_words: int = 4242,
        max_tokens: int = 4242,
        temperature: float = 0,
        clear_kv_cache=False,
        inference_type: INFERENCE_TYPE = INFERENCE_TYPE.CUSTOMER_INTERACTION,
    ) -> str:
        try:
            if inference_type == INFERENCE_TYPE.CUSTOMER_INTERACTION:
                formatted_prompt = PROMPT_TEMPLATE.format(
                    text=prompt,
                    inst=self.customerPromptFastmodel,
                    example=EXAMPLE,
                    query=query,
                    additional_information=query_additional_info,
                )

            if self.is_api:
                response = requests.post(
                    self.endpoint + f"?clear_kv_cache={clear_kv_cache}",
                    json={
                        "prompt": formatted_prompt,
                        "n_predict": n_words,
                        "stop": [
                            "</s>",
                            "</",
                            "</s",
                            "</ .",
                            "[/INST",
                            "[/INST]",
                            "[INST]",
                            "[INST" "\n",
                            "<END",
                            "<END>",
                            "\n\n",
                        ],
                        "stream": False,
                        "temperature": temperature,
                        "top_k": 1,
                        "top_p": 1,
                        "state_name": None,
                        "max_tokens": max_tokens,
                        # "clear_kv_cache": clear_kv_cache
                    },
                    timeout=120,
                )
                output = response.json()
                print(f"LLama cpp Fast model Session Response: {output}")
                if output and "choices" in output.keys():
                    return output["choices"][0]["text"]
                else:
                    raise Exception("Model prediction via api failed: ", output)
        except Exception as e:
            print(f"fast model api failed: {e}")

    async def apredict(
        self,
        prompt: str,
        query: str,
        query_additional_info: str = "",
        n_words: int = 4242,
        max_tokens: int = 4242,
        temperature: float = 0,
        clear_kv_cache=False,
        inference_type: INFERENCE_TYPE = INFERENCE_TYPE.CUSTOMER_INTERACTION,
    ) -> str:
        try:
            if inference_type == INFERENCE_TYPE.CUSTOMER_INTERACTION:
                formatted_prompt = PROMPT_TEMPLATE.format(
                    text=prompt,
                    inst=self.customerPromptFastmodel,
                    example=EXAMPLE,
                    query=query,
                    additional_information=query_additional_info,
                )

            if self.is_api:
                async with aiohttp.ClientSession() as session:
                    async with session.post(
                        self.endpoint + f"?clear_kv_cache={clear_kv_cache}",
                        json={
                            "prompt": formatted_prompt,
                            "n_predict": n_words,
                            "stop": [
                                "</s>",
                                "</",
                                "</s",
                                "</ .",
                                "[/INST",
                                "[/INST]",
                                "[INST]",
                                "[INST" "\n",
                                "<END",
                                "<END>",
                                "\n\n",
                            ],
                            "stream": False,
                            "temperature": temperature,
                            "top_k": 1,
                            "top_p": 1,
                            "state_name": None,
                            "max_tokens": max_tokens,
                            # "clear_kv_cache": clear_kv_cache
                        },
                    ) as response:
                        if response.status != 200:
                            # print(await response.text())
                            raise Exception("failed to update call")
                        else:
                            output = await response.json()
                            # self.logger.info(f"LLama cpp Fast model Session Response: {output}")
                            print(f"LLama cpp Fast model Session Response: {output}")
                            if output and "choices" in output.keys():
                                return output["choices"][0]["text"]
                            else:
                                raise Exception("Model prediction via api failed: ", output)
        except Exception as e:
            print(e)

In [6]:
llm = MistralModel(customerPromptFastmodel=fastModelLevel1PromptGpt)

In [7]:
sample_dialogue = """Transcript for Conversation ID: U3A1aFOfUOaRwxi2AjWnF
CUSTOMER: Hello there
AGENT: Hello Michael. Thanks for coming down to meet me today. The weather is really hot today.
CUSTOMER: Hey Ali, yeah, it's been quite warm lately.
CUSTOMER: By the way, have you tried the new cold brew from this coffee shop?
CUSTOMER: It's seriously refreshing on a day like this.
CUSTOMER: 🙅
CUSTOMER: [❤️=0]
AGENT: Oh, really? Yeah, I've never tried. Would you like to get a drink? I can get a cold brew for you.
CUSTOMER: That sounds like a great idea, thank you!
CUSTOMER: Could really use something cool right about now."""

In [8]:
query_list = []

for key1, value1 in name_mapping.items():
    for key2, value2 in value1.items():
        query_list.append(key2 + " " + value2)


tmp_query_list = query_list[:10]

In [9]:
# llm.predict(conversation_log)

In [10]:
task_list = []
for query in tmp_query_list:
    if query[:2] == "1.":
        task = llm.apredict(sample_dialogue, query, query_additional_info=product_info)
    else:
        task = llm.apredict(sample_dialogue, query)
    task_list.append(task)

In [11]:
import asyncio


# Run coroutines using asyncio.gather()
async def run_tasks():
    return await asyncio.gather(*task_list)


# Run event loop (This method is used because jupyter notebook itself is a event loop)
all_responses = await run_tasks()  # Takes about 1 min 10 seconds

LLama cpp Fast model Session Response: {'id': 'cmpl-86064d7e-63c3-4e69-9377-271c85cd248a', 'object': 'text_completion', 'created': 1714721431, 'model': 'mistral-7b-instruct-v0.2.Q6_K.gguf', 'choices': [{'text': 'ASSESSMENT: No\nREASON: The agent did not use any wrong insurance terms during the conversation.\nEXTRACT: N/A\nIMPROVEMENT: N/A', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1339, 'completion_tokens': 43, 'total_tokens': 1382}}
LLama cpp Fast model Session Response: {'id': 'cmpl-05e8bc87-5b9c-4a49-8630-5b3f6b48b46a', 'object': 'text_completion', 'created': 1714721432, 'model': 'mistral-7b-instruct-v0.2.Q6_K.gguf', 'choices': [{'text': 'ASSESSMENT: No\nREASON: In this conversation, the agent did not mention any comparison with competitors or their offerings.\nEXTRACT: "Oh, really? Yeah, I\'ve never tried."\nIMPROVEMENT: Instead of agreeing with the customer, the agent could have mentioned how FWD\'s insurance plans offer unique benefits c

In [12]:
all_responses

['ASSESSMENT: No\nREASON: The agent did not use any wrong insurance terms during the conversation.\nEXTRACT: N/A\nIMPROVEMENT: N/A',
 'ASSESSMENT: No\nREASON: The agent did not mention anything about insurance during the conversation.\nEXTRACT: N/A\nIMPROVEMENT: Instead of focusing on the weather, the agent could have started the conversation by asking about Michael\'s current insurance coverage and explaining how FWD\'s insurance plans can help protect him and his family from financial risks. For example: "Hi Michael, it\'s great to finally meet you. Before we enjoy our drinks, may I ask if you currently have any insurance coverage? Our FWD Set for Life plan offers comprehensive protection for life, health, and investments, allowing you to secure your future and safeguard your loved ones."',
 'ASSESSMENT: No\nREASON: The agent did not mention anything about the coverage of the policy during the conversation.\nEXTRACT: "Oh, really? Yeah, I’ve never tried."\nIMPROVEMENT: Instead of agre

In [15]:
tmp_query_list

['1.1.1 Did_AGENT_use_any_wrong_insurance_terms (reverse)',
 '1.1.2 Did_AGENT_explain_the_value_of_insurance_in_mitigating_risk',
 '1.2.1 Did AGENT explain coverage of the policy?',
 '1.2.2 Did AGENT explain features of the policy?',
 '1.2.3 Did AGENT give any wrong response regarding policy exclusions? (reverse)',
 '1.2.4 Did AGENT compare product features with those from competitors when necessary?',
 "1.2.5 Did AGENT explain how the product fits CUSTOMER's unique situation?",
 '1.3.1 Did AGENT give an example or analogy of how the policy is in line with current social/economic/market situation?',
 '2.1.1 Did AGENT open the conversation with professional and friendly greeting?',
 '2.1.2 Did AGENT initial small talk or show genuine interest in CUSTOMER to build rapport?']

In [16]:
for i, reply in zip(tmp_query_list, all_responses):
    print(i)
    print(reply)

1.1.1 Did_AGENT_use_any_wrong_insurance_terms (reverse)
ASSESSMENT: No
REASON: The agent did not use any wrong insurance terms during the conversation.
EXTRACT: N/A
IMPROVEMENT: N/A
1.1.2 Did_AGENT_explain_the_value_of_insurance_in_mitigating_risk
ASSESSMENT: No
REASON: The agent did not mention anything about insurance during the conversation.
EXTRACT: N/A
IMPROVEMENT: Instead of focusing on the weather, the agent could have started the conversation by asking about Michael's current insurance coverage and explaining how FWD's insurance plans can help protect him and his family from financial risks. For example: "Hi Michael, it's great to finally meet you. Before we enjoy our drinks, may I ask if you currently have any insurance coverage? Our FWD Set for Life plan offers comprehensive protection for life, health, and investments, allowing you to secure your future and safeguard your loved ones."
1.2.1 Did AGENT explain coverage of the policy?
ASSESSMENT: No
REASON: The agent did not me

In [ ]:
reply